# Making the Reference Genome more easily usable

> "Preparing genomic DNA for pre-training."

## Steps

1. Identify the latest genomic dna file in the downloaded data
2. Extract the accession sequence records from the reference genome annotation
3. Extract the DNA feature sequences, write to csv
4. Extract the mRNA feature sequences, write to csv
5. Extract the protein feature sequences, write to csv

### 0. Setup

In [1]:
#| default_exp pre_training.transform

In [2]:
#| hide
from nbdev.showdoc import *

In [23]:
#| export
from pathlib import Path
from Bio import SeqIO, SeqRecord, SeqFeature, Seq
from tqdm.auto import tqdm
from yaml import safe_load
import gzip
import pandas as pd
import typing
from tqdm.notebook import tqdm

from llm_mito_scanner.data.download import load_config

In [4]:
#| hide
config = load_config()

In [5]:
#| hide
data_path = Path(config.get("data_path"))
data_path_raw = data_path / "raw"
data_path_raw.exists()

True

### 1. Identify the latest genomic reference file

In [6]:
#| hide
annotations = [d for d in data_path_raw.iterdir() if d.is_dir()]
annotations

[Path('/mnt/e/Data/llm-mito-scanner-data/data/raw/GCF_000001405.40_GRCh38.p14')]

In [7]:
#| export
def get_latest_annotation_path(
        path: Path # Raw data path
        ) -> Path: # Path of the latest annotation
    "Get the latest annotation path."
    annotations = [d for d in path.iterdir() if d.is_dir()]
    annotations_df = pd.DataFrame(annotations, columns=['path'])
    annotations_df.loc[:, 'accession'] = annotations_df.path.apply(lambda p: p.name)
    annotations_df.loc[:, 'accession_prefix'] = annotations_df.accession.apply(
        lambda acc: acc.split(".", 1)[0]
    )
    annotations_df.sort_values("accession_prefix", inplace=True, ascending=False)
    return annotations_df.iloc[0, 0]

In [8]:
#| hide
latest_annotation_path = get_latest_annotation_path(data_path_raw)
latest_annotation_path

Path('/mnt/e/Data/llm-mito-scanner-data/data/raw/GCF_000001405.40_GRCh38.p14')

### 2. Extract the accession sequence records from the reference genome annotation

In [9]:
#| export
def get_latest_feature_table_path(
        path: Path # Latest annotation path
        ) -> Path: # Path of the feature table
    "Return latest feature table path."
    return next(path.glob("*_feature_table.txt.gz"), None)


def read_latest_feature_table(
        path: Path # Feature table path
        ) -> pd.DataFrame: # Dataframe of features
    "Return latest feature table."
    return pd.read_csv(
        path,
        compression="gzip",
        sep='\t'
    )

In [10]:
#| hide
# Where is the information for genes?
feature_table = read_latest_feature_table(
    get_latest_feature_table_path(latest_annotation_path)
)

/tmp/ipykernel_749/2127894616.py:13: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(


In [11]:
#| hide
feature_table['# feature'].value_counts()

# feature
CDS              145312
mRNA             144434
gene              67127
ncRNA             36872
misc_RNA          14995
precursor_RNA      2139
tRNA                691
V_segment           664
J_segment           128
rRNA                 80
D_segment            61
C_region             44
Name: count, dtype: int64

In [12]:
#| hide
feature_table.head()

,# feature,class,assembly,assembly_unit,seq_type,chromosome,genomic_accession,start,end,strand,product_accession,non-redundant_refseq,related_accession,name,symbol,GeneID,locus_tag,feature_interval_length,product_length,attributes
0,gene,transcribed_pseudogene,GCF_000001405.40,Primary Assembly,chromosome,1,NC_000001.11,11874,14409,+,NaN,NaN,NaN,DEAD/H-box helicase 11 like 1 (pseudogene),DDX11L1,100287102,NaN,2536,NaN,pseudo
1,misc_RNA,NaN,GCF_000001405.40,Primary Assembly,chromosome,1,NC_000001.11,11874,14409,+,NR_046018.2,NaN,NaN,DEAD/H-box helicase 11 like 1 (pseudogene),DDX11L1,100287102,NaN,1652,1652.0,pseudo
2,gene,transcribed_pseudogene,GCF_000001405.40,Primary Assembly,chromosome,1,NC_000001.11,14362,29370,-,NaN,NaN,NaN,"WASP family homolog 7, pseudogene",WASH7P,653635,NaN,15009,NaN,pseudo
3,misc_RNA,NaN,GCF_000001405.40,Primary Assembly,chromosome,1,NC_000001.11,14362,29370,-,NR_024540.1,NaN,NaN,"WASP family homolog 7, pseudogene",WASH7P,653635,NaN,1769,1786.0,pseudo
4,gene,miRNA,GCF_000001405.40,Primary Assembly,chromosome,1,NC_000001.11,17369,17436,-,NaN,NaN,NaN,microRNA 6859-1,MIR6859-1,102466751,NaN,68,NaN,NaN


In [13]:
#| export
def get_genomic_fasta_path(
        annotation_path: Path # Annotation path,
        ) -> Path: # Fasta path
    return next(annotation_path.glob("*[!from]_genomic.fna.gz"), None)

In [14]:
#| hide
latest_genomic_fasta_path = get_genomic_fasta_path(latest_annotation_path)

In [24]:
#| export
def extract_accession_sequence_records(
        gzip_fasta_path: Path,
        write_path: Path,
        expected_accessions: int = None
):
    pbar = None
    if isinstance(expected_accessions, int):
        pbar = tqdm(total=expected_accessions, ncols=80, leave=True)
    with gzip.open(str(gzip_fasta_path.resolve()), mode='rt') as f:
        for record in SeqIO.parse(f, "fasta"):
            record_write_path = write_path / f"{record.id}.fasta"
            if not record_write_path.exists():
                SeqIO.write(record, record_write_path, "fasta")
            if pbar is not None:
                pbar.update(1)
    if pbar is not None:
        pbar.close()

In [25]:
#| hide
accession_path = data_path_raw / "genomic_accessions"
if not accession_path.exists():
    accession_path.mkdir()

extract_accession_sequence_records(
    latest_genomic_fasta_path,
    accession_path,
    feature_table.genomic_accession.unique().size
)

  0%|                                                   | 0/538 [00:00<?, ?it/s]

In [27]:
#| export
def get_accession_sequence_record(
        accession_path: str,
) -> SeqRecord:
    with accession_path.open("r") as f:
        return next(SeqIO.parse(f, "fasta"), None)

In [28]:
#| hide
example_accession = get_accession_sequence_record(
    accession_path / "NC_000001.11.fasta"
)

In [29]:
#| hide
example_accession

SeqRecord(seq=Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN'), id='NC_000001.11', name='NC_000001.11', description='NC_000001.11 Homo sapiens chromosome 1, GRCh38.p14 Primary Assembly', dbxrefs=[])

In [37]:
#| hide
# Some validation we can find everything in the feature table
assert len(set(feature_table.genomic_accession.unique().tolist()) - set([f.stem for f in accession_path.glob("*.fasta")])) == 0

### 3. Extract the DNA feature sequences, write to csv

In [53]:
#| export
def yield_dna_feature_sequences(
        feature_table: pd.DataFrame, # Feature DataFrame
        accessions_path_parent: Path
        ) -> typing.Generator[typing.Tuple[int, str], None, None]: # Yield nucleotide strings denoting the feature
    "For each accession in the feature table, yield the index and sequence of the features."
    for accession in feature_table.genomic_accession.unique():
        accession_features = feature_table[feature_table.genomic_accession == accession]
        accession_path = accessions_path_parent / f"{accession}.fasta"
        if not accession_path.exists():
            raise FileNotFoundError(accession_path)
        accession_sequence_record = get_accession_sequence_record(accession_path)
        for idx, feature in accession_features.iterrows():
            feature_start = feature.start
            feature_end = feature.end
            feature_strand = feature.strand
            feature_interval_length = feature.feature_interval_length
            feature_sequence_record = accession_sequence_record[feature_start - 1: feature_end]
            if feature_strand == "-":
                feature_sequence_record = feature_sequence_record.reverse_complement()
            feature_sequence = feature_sequence_record.seq
            # assert len(feature_sequence) == feature_interval_length
            yield idx, str(feature_sequence)

In [54]:
#| hide
feature_table.shape[0], feature_table.drop_duplicates(subset=["genomic_accession", "start", "end", "strand", "GeneID"]).shape[0]

(412547, 181676)

In [55]:
#| hide
example_feature_sequence_generator = yield_dna_feature_sequences(
    feature_table,
    accession_path
)

(example_feature_idx, example_feature_sequence) = next(example_feature_sequence_generator)
(example_feature_idx_2, example_feature_sequence_2) = next(example_feature_sequence_generator)
(example_feature_idx_3, example_feature_sequence_3) = next(example_feature_sequence_generator)

In [56]:
#| hide
feature_table.iloc[[example_feature_idx, example_feature_idx_2, example_feature_idx_3], :]

,# feature,class,assembly,assembly_unit,seq_type,chromosome,genomic_accession,start,end,strand,product_accession,non-redundant_refseq,related_accession,name,symbol,GeneID,locus_tag,feature_interval_length,product_length,attributes
0,gene,transcribed_pseudogene,GCF_000001405.40,Primary Assembly,chromosome,1,NC_000001.11,11874,14409,+,NaN,NaN,NaN,DEAD/H-box helicase 11 like 1 (pseudogene),DDX11L1,100287102,NaN,2536,NaN,pseudo
1,misc_RNA,NaN,GCF_000001405.40,Primary Assembly,chromosome,1,NC_000001.11,11874,14409,+,NR_046018.2,NaN,NaN,DEAD/H-box helicase 11 like 1 (pseudogene),DDX11L1,100287102,NaN,1652,1652.0,pseudo
2,gene,transcribed_pseudogene,GCF_000001405.40,Primary Assembly,chromosome,1,NC_000001.11,14362,29370,-,NaN,NaN,NaN,"WASP family homolog 7, pseudogene",WASH7P,653635,NaN,15009,NaN,pseudo


In [57]:
#| hide
(feature_table.end - feature_table.start).describe()

count    4.125470e+05
mean     6.561172e+04
std      1.277448e+05
min      7.000000e+00
25%      6.873000e+03
50%      2.335000e+04
75%      6.994100e+04
max      2.473619e+06
dtype: float64

In [58]:
#| export
def write_dna_feature_sequences(
        feature_generator: typing.Generator[typing.Tuple[int, str], None, None], # Feature sequence generator
        file_path: Path, # Where we're writing to
        feature_num: int = None, # Number of features we expect to write, creates a progress bar
        batch_limit_len: int = 5000 # Number of features we gather before we write
        ):
    "Get features and their sequences, write to disk."
    batch_size = 0
    batch = []
    write_header = ~file_path.exists()
    pbar = None
    if isinstance(feature_num, int):
        pbar = tqdm(total=feature_num, position=0, ncols=80)
        pbar.set_description("Collecting")
    for feature_idx, feature_sequence in feature_generator:
        batch.append([feature_idx, feature_sequence])
        batch_size += 1
        if batch_size >= batch_limit_len:
            pbar.set_description("Writing")
            pd.DataFrame(
                batch,
                columns=['idx', 'sequence'],
            ).to_csv(
                file_path, 
                header=write_header, 
                index=False,
                mode="w+" if write_header else "a")
            if pbar is not None:
                pbar.update(batch_size)
                pbar.set_description("Collecting")
            write_header = False
            batch = []
            batch_size = 0
    if pbar is not None:
        pbar.set_description("Final Write")
    pd.DataFrame(
        batch,
        columns=['idx', 'sequence']
    ).to_csv(
        file_path, 
        header=write_header, 
        index=False,
        mode="w+" if write_header else "a")
    if pbar is not None:
        pbar.update(len(batch))
        pbar.close()

In [59]:
#| hide
feature_table_dedupe = feature_table.drop_duplicates(subset=["genomic_accession", "start", "end", "strand", "GeneID"])
# Remove features already written
pretrain_path = data_path / "pre-train"
if not pretrain_path.exists():
    pretrain_path.mkdir(parents=True)

dna_feature_file_path = pretrain_path / "dna_features.csv"

feature_table_to_write = feature_table_dedupe
if dna_feature_file_path.exists():
    written_feature_idx = pd.read_csv(dna_feature_file_path, usecols=['idx']).idx.unique()

    feature_table_to_write = feature_table_dedupe[feature_table_dedupe.index.isin(written_feature_idx) == False]
feature_table_to_write.shape[0]

101676

In [60]:
#| hide
dna_feature_generator = yield_dna_feature_sequences(
    feature_table_to_write,
    accession_path
)
for _ in dna_feature_generator:
    pass

In [61]:
#| hide
if feature_table_to_write.shape[0] > 0:
    dna_feature_generator = yield_dna_feature_sequences(
        feature_table_to_write,
        accession_path
    )
    write_dna_feature_sequences(
        dna_feature_generator,
        dna_feature_file_path,
        feature_num=feature_table_to_write.shape[0]
    )

  0%|                                                | 0/101676 [00:00<?, ?it/s]

### 4. Extract the mRNA feature sequences, write to csv

The mRNA sequences we have from NCBI remove introns. We want sequences that line up 1-to-1 with the gene the mRNA came from.

So we need to be able to;

1. Read mRNA sequences
2. Identify intronic sequences
3. Align with the source gene
4. Insert intron sequences to the sequence
5. Write transformed sequences for pre-training

In [64]:
#| export
def get_latest_rna_annotation_file(annotation_path: Path # Annotation path,
        ) -> Path: # Fasta path
    # GCF_000001405.40_GRCh38.p14_rna_from_genomic.fna.gz
    return next(annotation_path.glob("*_rna_from_genomic.fna.gz"), None)

In [66]:
#| hide
latest_rna_annotation_file = get_latest_rna_annotation_file(latest_annotation_path)
latest_rna_annotation_file

Path('/mnt/e/Data/llm-mito-scanner-data/data/raw/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_rna_from_genomic.fna.gz')

In [68]:
#| hide
with gzip.open(str(latest_rna_annotation_file.resolve()), mode='rt') as f:
    for example_rna_record in SeqIO.parse(f, "fasta"):
        break
example_rna_record

SeqRecord(seq=Seq('CTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTATTTGCTGT...CTG'), id='lcl|NC_000001.11_miscrna_NR_046018.2_1', name='lcl|NC_000001.11_miscrna_NR_046018.2_1', description='lcl|NC_000001.11_miscrna_NR_046018.2_1 [gene=DDX11L1] [db_xref=GeneID:100287102] [product=DEAD/H-box helicase 11 like 1 (pseudogene)] [pseudo=true] [transcript_id=NR_046018.2] [location=join(11874..12227,12613..12721,13221..14409)] [gbkey=misc_RNA]', dbxrefs=[])

In [79]:
#| hide
example_rna_record.name

'lcl|NC_000001.11_miscrna_NR_046018.2_1'

In [80]:
#| hide
example_rna_record.id

'lcl|NC_000001.11_miscrna_NR_046018.2_1'

In [78]:
#| hide
example_rna_record.description

'lcl|NC_000001.11_miscrna_NR_046018.2_1 [gene=DDX11L1] [db_xref=GeneID:100287102] [product=DEAD/H-box helicase 11 like 1 (pseudogene)] [pseudo=true] [transcript_id=NR_046018.2] [location=join(11874..12227,12613..12721,13221..14409)] [gbkey=misc_RNA]'

In [86]:
#| export
def parse_ncbi_fasta_description(seq_record: SeqRecord) -> typing.Tuple[str, dict[str, str]]:
    description = seq_record.description.split("|")[-1]
    description_split = description.split(" [")
    uid, features = description_split[0], description_split[1:]
    print(uid, features)
    features = [f.replace("[", "").replace("]", "").split("=") for f in features]
    # parse features
    features_dict = {
        f[0]: f[1] for f in features  
    }
    return uid, features_dict

In [90]:
#| hide
example_rna_record_uid, example_rna_record_description_features = parse_ncbi_fasta_description(example_rna_record)
example_rna_record_uid, example_rna_record_description_features

NC_000001.11_miscrna_NR_046018.2_1 ['gene=DDX11L1]', 'db_xref=GeneID:100287102]', 'product=DEAD/H-box helicase 11 like 1 (pseudogene)]', 'pseudo=true]', 'transcript_id=NR_046018.2]', 'location=join(11874..12227,12613..12721,13221..14409)]', 'gbkey=misc_RNA]']


('NC_000001.11_miscrna_NR_046018.2_1',
 {'gene': 'DDX11L1',
  'db_xref': 'GeneID:100287102',
  'product': 'DEAD/H-box helicase 11 like 1 (pseudogene)',
  'pseudo': 'true',
  'transcript_id': 'NR_046018.2',
  'location': 'join(11874..12227,12613..12721,13221..14409)',
  'gbkey': 'misc_RNA'})

In [92]:
#| export
def get_rna_sequence_locations(description_features: dict[str, str]) -> typing.List[typing.Tuple[int, int]]:
    location_str = description_features.get("location").replace(
        "join(", ""
    ).replace(")", "")
    location_str_split = location_str.split(",")
    locations = [tuple(map(int, l.split(".."))) for l in location_str_split]
    return locations

In [93]:
#| hide
example_rna_record_locations = get_rna_sequence_locations(
    example_rna_record_description_features
)
example_rna_record_locations

[(11874, 12227), (12613, 12721), (13221, 14409)]

### 5. Extract the protein feature sequences, write to csv

In [63]:
#| hide
import nbdev; nbdev.nbdev_export()